# [Regression task](https://www.kaggle.com/competitions/tutors-lessons-prices-prediction)


## My code

In [2]:
import sklearn
import pandas as pd

In [4]:
train = pd.read_excel("train.xlsx")
test = pd.read_excel("test.xlsx")

train["ФИО"].value_counts()

ФИО
Репетитор по информатике Макаров Иван Михайлович - фотография             9
Репетитор по информатике Минак Андрей Геннадьевич - фотография            9
Репетитор по информатике Маевская Ольга Витальевна - фотография           9
Репетитор по информатике Терехов Артем Владимирович - фотография          9
Репетитор по информатике Мышко Юлия Юрьевна - фотография                  9
                                                                         ..
Репетитор по математике  Развозжаева Алла Викторовна - фотография         1
Репетитор по математике  Суслова Александра Александровна - фотография    1
Репетитор по математике  Болат Санжар  - фотография                       1
Репетитор по математике  Ильдусова Руфина Эдуардовна - фотография         1
Репетитор по математике  Акперов Нурлан Кадырханович - фотография         1
Name: count, Length: 3817, dtype: int64

In [ ]:
zero_expel = ["Unnamed: 0"]  # Expel due to non-use
first_expel = ["ФИО", "description", "experience_desc"]  # Expel due to low correlation

train = train.drop(columns=zero_expel + first_expel)
test = test.drop(columns=zero_expel + first_expel)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   предмет           4548 non-null   object 
 1   tutor_head_tags   4548 non-null   object 
 2   tutor_rating      3959 non-null   float64
 3   tutor_reviews     4548 non-null   int64  
 4   Education_1       4548 non-null   object 
 5   Desc_Education_1  4472 non-null   object 
 6   Education_2       781 non-null    object 
 7   Desc_Education_2  777 non-null    object 
 8   Education_3       106 non-null    object 
 9   Desc_Education_3  105 non-null    object 
 10  Education_4       16 non-null     object 
 11  Desc_Education_4  16 non-null     object 
 12  Ученая степень 1  76 non-null     object 
 13  Ученое звание 1   35 non-null     object 
 14  status            4535 non-null   object 
 15  experience        4523 non-null   object 
 16  Desc_Education_5  3 non-null      object 


In [6]:
# Professionality normalize
education_cols = [col for col in train.columns if col.startswith('Education_')]
degree_cols = [col for col in train.columns if col.startswith('Ученая степень ')]
title_cols = [col for col in train.columns if col.startswith('Ученое звание ')]

train['education_count'] = train[education_cols].notna().sum(axis=1)
test['education_count'] = test[education_cols].notna().sum(axis=1)
train['degree_count'] = train[degree_cols].notna().sum(axis=1)
test['degree_count'] = test[degree_cols].notna().sum(axis=1)
train['title_count'] = train[title_cols].notna().sum(axis=1)
test['title_count'] = test[title_cols].notna().sum(axis=1)

education_expel = [f"Education_{i}" for i in range(1, 7)]
education_expel += [f"Desc_Education_{i}" for i in range(1, 7)]
degree_expel = [f"Ученая степень {i}" for i in range(1, 3)]
title_expel = [f"Ученое звание {i}" for i in range(1, 3)]

train = train.drop(columns=education_expel + degree_expel + title_expel)
test = test.drop(columns=education_expel + degree_expel + title_expel)

# Numeric refreshion
train['experience'] = train['experience'].str.extract('(\\d+)')
test['experience'] = test['experience'].str.extract('(\\d+)')
train['experience'] = pd.to_numeric(train['experience'])
test['experience'] = pd.to_numeric(test['experience'])

train

,предмет,tutor_head_tags,tutor_rating,tutor_reviews,status,experience,categories,mean_price,education_count,degree_count,title_count
0,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",5.0,2,Университетский преподаватель,31.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1,0,0
1,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,Школьный преподаватель,8.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",11.666667,2,0,0
2,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Школьный преподаватель,11.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,3,0,0
3,matematika,"['Школьная программа', 'Математический анализ'...",4.0,19,Частный преподаватель,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",0.116667,1,0,0
4,matematika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Частный преподаватель,19.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4543,matematika,"['Школьная программа', 'Математический анализ'...",4.0,11,Частный преподаватель,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",16.666667,1,0,0
4544,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Школьный преподаватель,30.0,['Школьники с 5 по 9 класс'],33.333333,1,0,0
4545,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Студент,6.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,1,0,0
4546,informatika,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,Студент,1.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",9.722222,1,0,0


In [ ]:
# Numeric refreshion of vectors
train = train.join(pd.get_dummies(train["предмет"]))
test = test.join(pd.get_dummies(train["предмет"]))
train = train.drop(columns=["предмет"])
test = test.drop(columns=["предмет"])

train = train.join(train['status'].str.get_dummies(sep=','))
test = test.join(test['status'].str.get_dummies(sep=','))
train = train.drop(columns=["status"])
test = test.drop(columns=["status"])

train

,tutor_head_tags,tutor_rating,tutor_reviews,experience,categories,mean_price,education_count,degree_count,title_count,informatika,matematika,Аспирант или ординатор очной формы обучения,Студент,Университетский преподаватель,Частный преподаватель,Школьный преподаватель
0,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",5.0,2,31.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1,0,0,False,True,0,0,1,0,0
1,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",4.0,7,8.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",11.666667,2,0,0,False,True,0,0,0,0,1
2,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,11.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,3,0,0,True,False,0,0,0,0,1
3,"['Школьная программа', 'Математический анализ'...",4.0,19,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",0.116667,1,0,0,False,True,0,0,0,1,0
4,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,19.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",16.666667,1,0,0,False,True,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4543,"['Школьная программа', 'Математический анализ'...",4.0,11,5.0,"['Школьники с 1 по 4 класс', 'Школьники с 5 по...",16.666667,1,0,0,False,True,0,0,0,1,0
4544,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,30.0,['Школьники с 5 по 9 класс'],33.333333,1,0,0,True,False,0,0,0,0,1
4545,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,6.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",23.333333,1,0,0,True,False,0,1,0,0,0
4546,"['Школьная программа', 'Подготовка к ЕГЭ', 'По...",0.0,0,1.0,"['Школьники с 5 по 9 класс', 'Школьники с 10 п...",9.722222,1,0,0,True,False,0,1,0,0,0


In [8]:
# Numeric refreshion of matrix
from sklearn.preprocessing import MultiLabelBinarizer
import ast

train['tutor_head_tags'] = train['tutor_head_tags'].apply(ast.literal_eval)
test['tutor_head_tags'] = test['tutor_head_tags'].apply(ast.literal_eval)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(train['tutor_head_tags'])
tags_dummies_train = pd.DataFrame(tags_dummies, columns=mlb.classes_)
train = pd.concat([train, tags_dummies_train], axis=1)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(test['tutor_head_tags'])
tags_dummies_test = pd.DataFrame(tags_dummies, columns=mlb.classes_)
test = pd.concat([test, tags_dummies_test], axis=1)

train = train.drop('tutor_head_tags', axis=1)
test = test.drop('tutor_head_tags', axis=1)

train['categories'] = train['categories'].apply(ast.literal_eval)
test['categories'] = test['categories'].apply(ast.literal_eval)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(train['categories'])
tags_dummies_train = pd.DataFrame(tags_dummies, columns=mlb.classes_)
train = pd.concat([train, tags_dummies_train], axis=1)

mlb = MultiLabelBinarizer()
tags_dummies = mlb.fit_transform(test['categories'])
tags_dummies_test = pd.DataFrame(tags_dummies, columns=mlb.classes_)
test = pd.concat([test, tags_dummies_test], axis=1)

train = train.drop('categories', axis=1)
test = test.drop('categories', axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 50 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   tutor_rating                                                                     3959 non-null   float64
 1   tutor_reviews                                                                    4548 non-null   int64  
 2   experience                                                                       4523 non-null   float64
 3   mean_price                                                                       4548 non-null   float64
 4   education_count                                                                  4548 non-null   int64  
 5   degree_count                                                                     4548 non-null   int64  
 6   title_co

In [9]:
train = train.fillna(0)
test = test.fillna(0)

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4548 entries, 0 to 4547
Data columns (total 50 columns):
 #   Column                                                                           Non-Null Count  Dtype  
---  ------                                                                           --------------  -----  
 0   tutor_rating                                                                     4548 non-null   float64
 1   tutor_reviews                                                                    4548 non-null   int64  
 2   experience                                                                       4548 non-null   float64
 3   mean_price                                                                       4548 non-null   float64
 4   education_count                                                                  4548 non-null   int64  
 5   degree_count                                                                     4548 non-null   int64  
 6   title_co

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd

# Get union of all columns from both datasets (excluding target)
all_columns = pd.Index(
    train.columns.tolist() + 
    test.columns.tolist()
).unique().drop("mean_price", errors="ignore")

# PROPER DATA SPLITTING
# Training data
y_train = train["mean_price"]
X_train = train.reindex(columns=all_columns, fill_value=0)

# Test data (assuming test doesn't have target)
X_test = test.reindex(columns=all_columns, fill_value=0)

In [12]:
# Initialize and train model
lig_reg = LinearRegression()
lig_reg.fit(X_train, y_train)

# Make predictions
y_train_pred = lig_reg.predict(X_train)  # For training metrics
y_test_pred = lig_reg.predict(X_test)   # For actual test predictions

# Calculate TRAINING metrics
print("Training Metrics:")
print("MAE:", mean_absolute_error(y_train, y_train_pred))
print("MSE:", mean_squared_error(y_train, y_train_pred))
print("R²:", r2_score(y_train, y_train_pred))

Training Metrics:
MAE: 6.928806004392778
MSE: 91.72695392867826
R²: 0.16546088511505985


In [13]:
submition = pd.DataFrame(y_test_pred, columns=["mean_price"])
submition = submition.reset_index()

submition.to_csv("my_submit.csv", index=False)

## Deepseek method

Dataset Description.

You are given various information about tutors, you need to predict the average cost per minute of a given tutor (rub/ 1 min).

Files
- **train.xlsx** - the training set
- **test.xlsx** - the test set
- **sample_submission.csv** - a sample submission file in the correct format

Columns
- **предмет** - subject taught by the teacher
- **ФИО** - tutor's name
- **tutor_head_tags** - What level does the tutor prepare for?
- **tutor_rating** - Tutor's rating
- **tutor_reviews** - Number of reviews
- **description** - The "About me" field, which is filled by the tutor himself
- **experience_desc** - Description of the tutor experience
- **Education_i** - The university the tutor graduated from
- **Desc_Education_i** - Faculty, specialty, year of graduation and other information about education i
- **Ученая степень i** - Academic degree
- **Ученое звание i** - Academic title
- **categories** - Categories of students that the tutor is ready to take
- **mean_price** - Average cost of tutor services per minute

Consider `train.info()`:
~~~
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        4548 non-null   int64  
 1   предмет           4548 non-null   object 
 2   ФИО               4548 non-null   object 
 3   tutor_head_tags   4548 non-null   object 
 4   tutor_rating      3959 non-null   float64
 5   tutor_reviews     4548 non-null   int64  
 6   description       4547 non-null   object 
 7   experience_desc   2171 non-null   object 
 8   Education_1       4548 non-null   object 
 9   Desc_Education_1  4472 non-null   object 
 10  Education_2       781 non-null    object 
 11  Desc_Education_2  777 non-null    object 
 12  Education_3       106 non-null    object 
 13  Desc_Education_3  105 non-null    object 
 14  Education_4       16 non-null     object 
 15  Desc_Education_4  16 non-null     object 
 16  Ученая степень 1  76 non-null     object 
 17  Ученое звание 1   35 non-null     object 
 18  status            4535 non-null   object 
 19  experience        4523 non-null   object 
 20  Desc_Education_5  3 non-null      object 
 21  Desc_Education_6  1 non-null      object 
 22  Education_5       3 non-null      object 
 23  Education_6       1 non-null      object 
 24  Ученая степень 2  1 non-null      object 
 25  Ученое звание 2   1 non-null      object 
 26  categories        4548 non-null   object 
 27  mean_price        4548 non-null   float64
~~~

In [21]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import KFold

# Load data
train = pd.read_excel('train.xlsx')
test = pd.read_excel('test.xlsx')

# Initialize encoders with training data
mlb_tags = MultiLabelBinarizer()
mlb_categories = MultiLabelBinarizer()

def preprocess_data(df, mlb_tags, mlb_categories, is_train=False):
    # Convert string representations to lists
    df['tutor_head_tags'] = df['tutor_head_tags'].apply(eval)
    df['categories'] = df['categories'].apply(eval)
    
    # Fit/transform multi-label features
    if is_train:
        tags_dummies = mlb_tags.fit_transform(df['tutor_head_tags'])
        cats_dummies = mlb_categories.fit_transform(df['categories'])
    else:
        tags_dummies = mlb_tags.transform(df['tutor_head_tags'])
        cats_dummies = mlb_categories.transform(df['categories'])
    
    # Create DataFrames with consistent columns
    tags_df = pd.DataFrame(tags_dummies, columns=[f"tag_{x}" for x in mlb_tags.classes_])
    cats_df = pd.DataFrame(cats_dummies, columns=[f"cat_{x}" for x in mlb_categories.classes_])
    
    # Subject encoding with consistent categories
    subjects = pd.get_dummies(df['предмет'], prefix='subj')
    
    # Education features
    df['education_count'] = df[[c for c in df.columns if c.startswith('Education_')]].notna().sum(axis=1)
    
    # Experience processing
    df['experience'] = df['experience'].str.extract(r'(\d+)').astype(float)
    
    # Merge all features
    processed = pd.concat([
        df[['tutor_rating', 'tutor_reviews', 'education_count', 'experience']],
        subjects,
        tags_df,
        cats_df
    ], axis=1)
    
    # Fill missing values
    processed['tutor_rating'].fillna(processed['tutor_rating'].median())
    processed['experience'].fillna(processed['experience'].median())
    
    return processed

# Preprocess data with consistent encoding
train_proc = preprocess_data(train, mlb_tags, mlb_categories, is_train=True)
test_proc = preprocess_data(test, mlb_tags, mlb_categories)

# Align columns between train and test
missing_cols = set(train_proc.columns) - set(test_proc.columns)
for col in missing_cols:
    test_proc[col] = 0
test_proc = test_proc[train_proc.columns]

# Prepare target
y_train = train['mean_price'].values  # Keep original scale
X_train = train_proc
X_test = test_proc

# Train model without log transformation
model = CatBoostRegressor(
    iterations=2000,
    learning_rate=0.05,
    depth=7,
    cat_features=[c for c in X_train.columns if c.startswith('subj_')],
    verbose=0
)

# Cross-validated training
test_preds = np.zeros(X_test.shape[0])
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in kf.split(X_train):
    model.fit(X_train.iloc[train_idx], y_train[train_idx],
             eval_set=(X_train.iloc[val_idx], y_train[val_idx]),
             early_stopping_rounds=100)
    test_preds += model.predict(X_test) / kf.n_splits

# Create submission
pd.DataFrame({'mean_price': test_preds}).to_csv('submission.csv', index=False)

/home/andredon/projects/ADA/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['Студенты университетов группа'] will be ignored
  warnings.warn(


## Pipeline

In [9]:
import pandas as pd
import numpy as np
import math
import ast

from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

In [10]:
class PersonalizationDropper(BaseEstimator, TransformerMixin):
    """
    Drops personal or irrelevant columns from the dataset.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Columns that we don't need at all
        zero_expel = ["Unnamed: 0"]
        # Columns with low correlation or confidential data
        first_expel = ["ФИО", "description", "experience_desc"]
        
        X = X.drop(columns=zero_expel + first_expel, errors='ignore')
        return X


class ProfessionalityFeaturesExtractor(BaseEstimator, TransformerMixin):
    """
    Extracts features related to professional background, such as 
    the count of educational degrees and titles.
    Also cleans up the 'experience' feature.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Identify columns related to education, degrees, and titles
        education_cols = [col for col in X.columns if col.startswith('Education_')]
        degree_cols = [col for col in X.columns if col.startswith('Ученая степень ')]
        title_cols = [col for col in X.columns if col.startswith('Ученое звание ')]

        # Create new features counting non-null entries in each category
        X['education_count'] = X[education_cols].notna().sum(axis=1)
        X['degree_count']    = X[degree_cols].notna().sum(axis=1)
        X['title_count']     = X[title_cols].notna().sum(axis=1)

        # List columns to drop (since we've created count features, we can remove the originals)
        education_expel = [f"Education_{i}" for i in range(1, 7)] + \
                          [f"Desc_Education_{i}" for i in range(1, 7)]
        degree_expel = [f"Ученая степень {i}" for i in range(1, 3)]
        title_expel  = [f"Ученое звание {i}" for i in range(1, 3)]

        X = X.drop(columns=education_expel + degree_expel + title_expel, errors='ignore')

        # Clean the 'experience' column: extract digits and convert to numeric
        if 'experience' in X.columns:
            X['experience'] = X['experience'].str.extract(r'(\d+)')
            X['experience'] = pd.to_numeric(X['experience'])

        return X


class VectorFeaturesExtractor(BaseEstimator, TransformerMixin):
    """
    Transforms text/categorical columns 'предмет' and 'status' 
    into dummy/indicator variables.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Convert 'предмет' into dummy columns
        if 'предмет' in X.columns:
            X = X.join(pd.get_dummies(X['предмет']))
            X = X.drop(columns=['предмет'], errors='ignore')

        # Convert 'status' into dummy columns (multiple statuses separated by commas)
        if 'status' in X.columns:
            X = X.join(X['status'].str.get_dummies(sep=','))
            X = X.drop(columns=['status'], errors='ignore')

        return X
    

class MatrixFeaturesExtractor(BaseEstimator, TransformerMixin):
    """
    Uses MultiLabelBinarizer to transform 'tutor_head_tags' and 'categories'
    (stored as lists in string form) into dummy/indicator columns.
    """
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Safely convert 'tutor_head_tags' from string to list, then binarize
        if 'tutor_head_tags' in X.columns:
            X['tutor_head_tags'] = X['tutor_head_tags'].apply(ast.literal_eval)
            mlb_tags = MultiLabelBinarizer()
            tags_dummies = mlb_tags.fit_transform(X['tutor_head_tags'])
            tags_dummies_df = pd.DataFrame(tags_dummies, columns=mlb_tags.classes_, index=X.index)
            X = pd.concat([X, tags_dummies_df], axis=1)
            X = X.drop(columns=['tutor_head_tags'], errors='ignore')

        # Convert 'categories' similarly
        if 'categories' in X.columns:
            X['categories'] = X['categories'].apply(ast.literal_eval)
            mlb_cats = MultiLabelBinarizer()
            cats_dummies = mlb_cats.fit_transform(X['categories'])
            cats_dummies_df = pd.DataFrame(cats_dummies, columns=mlb_cats.classes_, index=X.index)
            X = pd.concat([X, cats_dummies_df], axis=1)
            X = X.drop(columns=['categories'], errors='ignore')

        return X


class Merger(BaseEstimator, TransformerMixin):
    """
    Ensures that any columns present in train or test are aligned. 
    Reindexes the dataframe to have all_columns (common in train + test).
    """
    def __init__(self, all_columns):
        self.all_columns = all_columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Reindex to ensure consistent columns and fill missing with 0
        X = X.reindex(columns=self.all_columns, fill_value=0)
        return X

In [11]:
train = pd.read_excel("train.xlsx")
test  = pd.read_excel("test.xlsx")

# Separate features (X) and target (y) in the training data
X_train = train.drop(columns=["mean_price"], errors='ignore')
y_train = train["mean_price"]

# Merge column lists from train and test so both sets share the same columns
all_columns = pd.Index(
    X_train.columns.tolist() + 
    test.columns.tolist()
).unique()  # We won't drop 'mean_price' here because we already dropped it from X_train


In [12]:
# Pipeline for preprocessing only
pipeline_preprocessing = Pipeline([
    ('personalization_drop', PersonalizationDropper()),    # 1) Drop unwanted columns
    ('professionality_features', ProfessionalityFeaturesExtractor()),  # 2) Create professional features
    ('vector_features', VectorFeaturesExtractor()),        # 3) Convert text columns to dummies
    ('matrix_features', MatrixFeaturesExtractor()),        # 4) Use MultiLabelBinarizer on lists
    ('merger', Merger(all_columns=all_columns)),           # 5) Align columns with train/test
    ('missing', KNNImputer()),                             # 6) Impute missing values
    ('scaler', StandardScaler()),                          # 7) Scale numerics
    ('encoding', OneHotEncoder(handle_unknown='ignore')) 
    # Note: The above OneHotEncoder will encode the columns output by the pipeline. 
    # If the dataset is large, consider using 'sparse=True' and final pipeline steps accordingly.
])

# Full pipeline: preprocessing + model
full_pipeline = Pipeline([
    ('preprocessing', pipeline_preprocessing),
    ('model', RandomForestRegressor(random_state=42))
])

In [13]:
# You can adjust this grid to see if you can improve performance
param_grid = {
    'model__n_estimators': [50, 100],
    'model__max_depth': [None, 5, 10]
}

grid_search = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    cv=3,                      # 3-fold cross-validation
    scoring='neg_mean_squared_error',
    verbose=1
)

# Fit on the training data
grid_search.fit(X_train, y_train)

# Best pipeline found
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best Parameters: {'model__max_depth': 10, 'model__n_estimators': 100}


In [ ]:
# Predict on the test set using the best found model
test_predictions = best_model.predict(test)

# Optionally, attach predictions to your test DataFrame
test['mean_price'] = test_predictions

# Show a few sample predictions
print(test[['mean_price']].head(10))

submition = pd.DataFrame(test, columns=["mean_price"])
submition = submition.reset_index()

submition.to_csv("my_submit.csv", index=False)

   predicted_mean_price
0             15.978527
1             14.047753
2             15.868569
3             14.727801
4             11.817369
5             15.868569
6             15.595045
7             18.364422
8             16.965173
9             14.859141
